In [7]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [8]:
train_data_dir = "TrainSet"
test_data_dir = "TestSet"

train_durations = []
for root, dirs, files in os.walk(train_data_dir):
    for file in files:
        if file.endswith('.wav'):
            file_path = os.path.join(root, file)
            audio, sr = librosa.load(file_path, sr=None)
            train_durations.append(len(audio))

test_durations = []
for root, dirs, files in os.walk(test_data_dir):
    for file in files:
        if file.endswith('.wav'):
            file_path = os.path.join(root, file)
            audio, sr = librosa.load(file_path, sr=None)
            test_durations.append(len(audio))

target_duration = max(max(train_durations), max(test_durations))


In [9]:
def load_audio_with_padding(file_path):
    audio, sr = librosa.load(file_path, sr=None,)
    if len(audio) < target_duration:
        pad_width = int(target_duration - len(audio))
        audio = np.pad(audio, pad_width=((0, pad_width)), mode='constant')
    return audio, sr


In [10]:
# Function to extract MFCC features from audio file

def extract_lpc(file_path, num_lpc=14):
    # Load audio file
    y, sr = load_audio_with_padding(file_path)
    
    # Extract LPC features
    lpc = librosa.core.lpc(y, order=num_lpc)
    
    return lpc

In [11]:
# Function to load data and extract features
def load_data(data_dir,):
    labels = []
    features = []

    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(root, file)
                label = int(os.path.basename(root))
                labels.append(label)
                lpc = extract_lpc(file_path)
                features.append(lpc.T)




    return features, labels

In [12]:


# Load training data
train_data_dir = "TrainSet"
X_train, y_train = load_data(train_data_dir)

# Load test data
test_data_dir = "TestSet"
X_test, y_test = load_data(test_data_dir)




In [13]:


X_train =  np.array( X_train)

X_test = np.array( X_test)

In [14]:
print(X_test.shape,X_train.shape)

(296, 15) (1900, 15)


In [15]:

# Define K values for KNN
k_values = [7, 11, 15, 20]

# Iterate over K values
for k in k_values:
    # Train KNN model
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)

    # Predict labels for test data
    y_pred = knn.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for K={k}: {accuracy}")


Accuracy for K=7: 0.5405405405405406
Accuracy for K=11: 0.5405405405405406
Accuracy for K=15: 0.5337837837837838
Accuracy for K=20: 0.4966216216216216


In [16]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# SVM with linear kernel using LPC features
svm_linear_lpc = SVC(kernel='linear')
svm_linear_lpc.fit(X_train, y_train)
y_pred_linear_lpc = svm_linear_lpc.predict(X_test)
accuracy_linear_lpc = accuracy_score(y_test, y_pred_linear_lpc)
print("Accuracy of SVM with linear kernel using LPC features:", accuracy_linear_lpc)

# SVM with polynomial kernel using LPC features
svm_poly_lpc = SVC(kernel='poly')
svm_poly_lpc.fit(X_train, y_train)
y_pred_poly_lpc = svm_poly_lpc.predict(X_test)
accuracy_poly_lpc = accuracy_score(y_test, y_pred_poly_lpc)
print("Accuracy of SVM with polynomial kernel using LPC features:", accuracy_poly_lpc)




Accuracy of SVM with linear kernel using LPC features: 0.6756756756756757
Accuracy of SVM with polynomial kernel using LPC features: 0.4222972972972973
